In [3]:
import numpy as np
from sklearn.datasets import load_sample_image
from PIL import Image
#import tensorflow.compat.v1 as tf
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import sys
import cv2
from scipy import signal



In [ ]:
im = Image.open(r"path to image goes here")
pix_val = list(im.getdata())

im = cv2.imread(r"path to image goes here")
#im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

#np.true_divide(im, np.amax(im))


hsv_img = tf.image.rgb_to_hsv(np.true_divide(im, np.amax(im)))
rgb_img = tf.image.hsv_to_rgb(hsv_img)
#with tf.Session() as sess: print(rgb_img.eval())
#with tf.Session() as sess: print(hsv_img.eval())


# sess = tf.Session()
# with sess.as_default():
#     cv2.imwrite('clr_img.jpg', rgb_img.eval())
#     cv2.imshow("image", rgb_img.eval())
#     cv2.waitKey()
   

split0, split1, split2 = tf.split(hsv_img, [1, 1, 1], 2)

sobel_x = np.array([[1, 2, 1],
                   [100, 100, 100],
                   [-1 , -2 , 1]])
                   

sobel_y = np.array([[-1, 100, 1],
                   [-2, 100, 2],
                   [-1, 100, 1]])

sobel_d1 = np.array([[100, 1, 2],
                   [-1, 100, 1],
                   [-2, -1, 100]])

sobel_d2 = np.array([[2, 1, 100],
                   [1, 100, -1],
                   [100, -1, -2]])


filters = np.concatenate([[sobel_x], [sobel_y], [sobel_d1], [sobel_d2]])
#filters = np.expand_dims(filters, -1)
filters = filters.transpose(1,2,0)

holder = split2.numpy()
result = holder[:, :, 0]


gradx = signal.convolve2d(result, sobel_x, boundary='fill', mode='same')
grady = signal.convolve2d(gradx, sobel_y, boundary='fill', mode='same')
gradd1 = signal.convolve2d(grady, sobel_d1, boundary='fill', mode='same')
gradd2 = signal.convolve2d(gradd1, sobel_d2, boundary='fill', mode='same')

gradd2 = np.expand_dims(gradd2, -1)
gradd2 = np.true_divide(gradd2, np.amax(gradd2))
gradd2 = gradd2.clip(min=0)


new_grad = tf.convert_to_tensor(gradd2)

hsv_complete = tf.concat([split0, split1, new_grad], 2)

rgb_img2 = tf.image.hsv_to_rgb(hsv_complete)


cv2.imwrite('clr_img.jpg',rgb_img2.numpy())
cv2.imshow("image", rgb_img2.numpy())
cv2.imwrite("nFiltered.png", 255*rgb_img2.numpy())
cv2.waitKey(0)
cv2.destroyWindow("image")








#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#hsv_ee = tf.image.rgb_to_hsv(im, name = "wayne")
#with tf.Session() as sess: print(hsv_ee.eval())